In [1]:
I_GPU = 1

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

In [3]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/kitti360'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/kitti360'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/kitti360'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/kitti360'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/kitti360'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/kitti360'  # ENGIE

In [ ]:
out_dir = '/workspace/projects/torch-points3d/outputs/'
result_dir = '/raid/dataset/pointcloud/data/kitti360/results'

# ('5cm', 'modelname',   out_dir + '____', '___'),

checkpoints = [
#     ('5cm', 'Res16UNet34', out_dir + '2021-11-09/09-10-01', 'xyzrgb'),
#     ('5cm', 'Res16UNet34', out_dir + '2021-11-16/09-40-20', 'xyzrgb_40ep_6k'),
#     ('5cm', 'Res16UNet34', out_dir + '2022-01-15/10-11-10', 'xyzrgb_60ep_6k'),
    
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes', out_dir + '2021-11-16/18-36-13', 'pyramid')
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes-interpolate', out_dir + '2021-12-11/15-48-03', 'interpolate_pyramid'),

#     ('5cm', 'Res16UNet34', out_dir + '2021-11-13/16-02-16', 'xyzrgb_60ep_12k'),
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes-interpolate', out_dir + '2022-01-21/15-39-56', 'interpolate_pyramid_60ep_12k'),
#     ('5cm', 'Res16UNet34-L4-early-cityscapes-interpolate', out_dir + '2022-01-19/13-36-32', 'interpolate_l4_60ep_12k'),
  
#     ('5cm', 'Res16UNet34-PointPyramid-early-max-cityscapes-interpolate', out_dir + '22022-01-23/22-15-32', 'interpolate_pyramid_max_12k'),

    
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes-interpolate-group1', out_dir + '2022-01-25/23-49-15', 'interpolate_pyramid_group1_6k'),
#     ('5cm', 'Res16UNet34-PointPyramid-early-fromscratch-interpolate', out_dir + '2022-01-25/16-02-03', 'interpolate_pyramid_fromscratch_6k'),
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes-interpolate', out_dir + '2022-02-08/14-07-32', 'interpolate_pyramid_704x188_6k'),
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes-interpolate-nogating', out_dir + '2022-02-08/14-05-14', 'interpolate_pyramid_nogating_6k'),
#     ('5cm', 'Res16UNet34-PointPyramid-early-cityscapes-interpolate', out_dir + '2022-02-07/19-24-19', 'interpolate_pyramid_nodynamicbatch_6k'),
#     ('5cm', 'Res16UNet34-PointPyramid-early-max-cityscapes-interpolate', out_dir + '2021-12rgb_6k'),
#     ('5cm', 'RGB_PointPyramid-cityscapes-interpolate', out_dir + '2022-02-11/14-44-03', 'rgb_6k'),
#     ('5cm', 'ResUNet34-PointPyramid-early-cityscapes-interpolate', out_dir + '2022-02-10/17-22/19-08-10', 'interpolate_pyramid_max_6k'),
#     ('5cm', 'Res16UNet34-mean-RGB-early', out_dir + '2022-02-11/09-21-56', 'avg--38-50', 'smaller-backbone_6k'),
    
#     ('5cm', 'Res16UNet34', out_dir + '2022-02-09/17-03-49', 'xyz_6k'),
       
]

# split = 'test'
split = 'val'
n_votes = 1
# sample_res = 3
sample_res = 6  # almost equivalent, faster but -0.15 mIoU, roughly
batch_size = 8
# batch_size = 12

for resolution, model_name, directory, exp_name in checkpoints:
    
    exp_name = model_name if exp_name is None else exp_name
    
    print(f'Run KITTI360 resolution={resolution} exp={exp_name}')
    
    overrides = [
        f'model_name={model_name}',
        f"checkpoint_dir={directory}",
        'voting_runs=1',
        'tracker_options.full_res=True',
        f'tracker_options.make_submission={split == "test"}',
        'precompute_multi_scale=False',
        'num_workers=4',
        f'batch_size={batch_size}',
        f'cuda={I_GPU}',
        'weight_name=last',
        f'+data.eval_sample_res={sample_res}'
    ]
    
    cfg = hydra_read(overrides, config_name='eval')
    
    OmegaConf.set_struct(cfg, False)
    
    from torch_points3d.trainer import Trainer
    
    trainer = Trainer(cfg)
    
    # Update the val and test transforms to match train transforms for inference-time augmentation
    trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
    
    if trainer._model.is_multimodal and exp_name != 'interpolate_pyramid_nodynamicbatch_6k':
        trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image
        trainer._dataset.test_dataset[0].transform_image.transforms[3].use_coverage = False
        trainer._dataset.test_dataset[0].transform_image.transforms[3].credit = int(1408 * 376 * 4 * 2)
    
    trainer._cfg.voting_runs = n_votes    
    
#     for i_try in range(4):
    for i_try in range(1):
        print(f'  run {i_try}...')
        trainer.eval(stage_name=split)
        torch.save(trainer._tracker._full_confusion_matrix, f'{result_dir}/{exp_name}_{split}_{resolution}_sample-res-{sample_res}_votes-{trainer._cfg.voting_runs}_{i_try}.pt')

Run KITTI360 resolution=5cm exp=avg-rgb_6k
invalid syntax (<string>, line 1)
  run 0...


  0%|          | 0/2400 [00:00<?, ?it/s]

In [ ]:
trainer._tracker._full_vote_miou

In [ ]:
cm = torch.load(f'{result_dir}/{exp_name}_{split}_{resolution}_{i_try}.pt')

In [ ]:
' & '.join([str(x) for x in np.round(cm.get_intersection_union_per_class()[0] * 100, 1)])

# Visualization of predictions

In [ ]:
val_dataset = trainer._dataset.val_dataset
# val_dataset.transform = trainer._dataset.train_dataset.transform
# val_dataset.transform_image = trainer._dataset.train_dataset.transform_image
# val_dataset.transform_image.transforms[3].use_coverage = False
# val_dataset.transform_image.transforms[3].credit = int(1408 * 376 * 4 * 2)

In [ ]:
# # Inference for Baseline
# from torch_geometric.data import Batch

# model = trainer._model.eval().cuda()

# with torch.no_grad():
#     pred_baseline = None
#     for i in range(1):
#         data = Batch.from_data_list([val_dataset[0]])
#         model.set_input(data, model.device)
#         model(data)
#         if pred_baseline is None:
#             pred_baseline = model.output.cpu()
#         else:
#             pred_baseline += model.output.cpu()
#     pred_baseline = pred_baseline.argmax(1)

# # Save the prediction for backup
# # torch.save(pred_baseline, '/raid/dataset/pointcloud/data/scannet/results/inferences_for_visualization/baseline_y.pt')

In [ ]:
# Only sample in the first window
idx_window = 0
num_samples = val_dataset.sampling_sizes[idx_window]
num_raw_points = val_dataset.window_raw_sizes[idx_window]
votes = torch.zeros(num_raw_points, val_dataset.num_classes)

# Inference for Baseline
from torch_geometric.data import Batch
from tqdm import tqdm

model = trainer._model.eval().cuda()

with torch.no_grad():
    for i_cylinder in tqdm(range(num_samples)):

        data = Batch.from_data_list([val_dataset[i_cylinder]])
        model.set_input(data, model.device)
        model(data)
        y = model.output.cpu()

        idx_seen = data.origin_id
        votes[idx_seen] += y
pred_baseline = votes.argmax(1)

In [ ]:
# torch.save(pred, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/baseline_y_pred_full')

In [ ]:
# Only sample in the first window
idx_window = 0
num_samples = val_dataset.sampling_sizes[idx_window]
num_raw_points = val_dataset.window_raw_sizes[idx_window]
votes = torch.zeros(num_raw_points, val_dataset.num_classes)

# Inference for Ours
from torch_points3d.core.multimodal.data import MMData, MMBatch
from tqdm import tqdm

model = trainer._model.eval().cuda()

with torch.no_grad():
    for i_cylinder in tqdm(range(num_samples)):

        mm_data = MMBatch.from_mm_data_list([val_dataset[i_cylinder]])
        model.set_input(mm_data, model.device)
        model(mm_data)
        y = model.output.cpu()
        idx_seen = mm_data.data.origin_id
        votes[idx_seen] += y
pred_ours = votes.argmax(1)

In [ ]:
data_gt = val_dataset.buffer[idx_window].data.clone()

data_baseline = val_dataset.buffer[idx_window].data.clone()
data_baseline.y = pred_baseline[data_baseline.origin_id]

data_ours = val_dataset.buffer[idx_window].data.clone()
data_ours.y = pred_ours[data_ours.origin_id]

In [ ]:
# torch.save(data_gt, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/data_gt.pt')
# torch.save(data_baseline, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/data_baseline.pt')
# torch.save(data_ours, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/data_ours.pt')

In [ ]:
from torch_points3d.visualization.multimodal_data import visualize_mm_data, hex_to_tensor
from torch_points3d.datasets.segmentation.kitti360_config import CLASS_NAMES, CLASS_COLORS
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.core.multimodal.image import ImageData

data_gt.mapping_index = torch.arange(data_gt.num_nodes)
mm_data_gt = MMData(data_gt, image=ImageData([SameSettingImageData()]))

data_baseline.mapping_index = torch.arange(data_baseline.num_nodes)
mm_data_baseline = MMData(data_baseline, image=ImageData([SameSettingImageData()]))

data_ours.mapping_index = torch.arange(data_ours.num_nodes)
mm_data_ours = MMData(data_ours, image=ImageData([SameSettingImageData()]))

In [ ]:
# torch.save(mm_data_gt, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/mm_data_gt.pt')
# torch.save(mm_data_baseline, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/mm_data_baseline.pt')
# torch.save(mm_data_ours, '/raid/dataset/pointcloud/data/kitti360/results/inference_for_visualization/mm_data_ours.pt')

In [ ]:
# Colorize in red the areas of interest for us 
idx_ideal = (mm_data_ours.y == mm_data_gt.y) & (mm_data_baseline.y != mm_data_gt.y)
mm_data_gt.data.rgb[idx_ideal] = torch.Tensor([1, 0, 0])

In [ ]:
visualize_mm_data(mm_data_gt, figsize=1000, pointsize=3, voxel=1, show_2d=False, front='map', class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)

In [ ]:
visualize_mm_data(mm_data_baseline, figsize=1000, pointsize=3, voxel=1, show_2d=False, front='map', class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)

In [ ]:
visualize_mm_data(mm_data_ours, figsize=1000, pointsize=3, voxel=1, show_2d=False, front='map', class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)

In [ ]:
in_range = (1060 < mm_data_ours.data.pos[:, 0]) & (mm_data_ours.data.pos[:, 0] < 1100)
mm_data_gt_range = mm_data_gt[in_range]
mm_data_baseline_range = mm_data_baseline[in_range]
mm_data_ours_range = mm_data_ours[in_range]

In [ ]:
visualize_mm_data(mm_data_gt_range, figsize=1000, pointsize=3, voxel=0.1, show_2d=False, class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)

In [ ]:
visualize_mm_data(mm_data_baseline_range, figsize=1000, pointsize=3, voxel=0.1, show_2d=False, class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)

In [ ]:
visualize_mm_data(mm_data_ours_range, figsize=1000, pointsize=3, voxel=0.1, show_2d=False, class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)

# Quick visualization of overlapping spheres

In [ ]:
loader = trainer._dataset._test_loaders[0]
trainer._dataset.test_dataset[0].transform = None
loader.num_workers = 0
for i, data in enumerate(loader):
    if i == 20:
        break

In [ ]:
from torch_points3d.visualization.multimodal_data import visualize_mm_data, hex_to_tensor, PALETTE
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.core.multimodal.image import SameSettingImageData, ImageData

colors = torch.cat([hex_to_tensor(x) for x in PALETTE]).view(-1, 3)

data.rgb = torch.ones_like(data.rgb)

for i_batch in range(loader.batch_size):
    start = data.__slices__['rgb'][i_batch]
    end = data.__slices__['rgb'][i_batch + 1]
    data.rgb[start:end] = data.rgb[start:end] * 0.5 + 0.5 * colors[i_batch % colors.shape[0]]

data.mapping_index = torch.arange(data.num_nodes)

In [ ]:
mm_data = MMData(data=data, image=ImageData([SameSettingImageData()]))
visualize_mm_data(mm_data, figsize=600, pointsize=3, voxel=0.5, show_2d=False)

In [ ]:
for mini_data in data.to_data_list():
    mm_data = MMData(data=mini_data, image=ImageData([SameSettingImageData()]))
    visualize_mm_data(mm_data, figsize=600, pointsize=3, voxel=0.5, show_2d=False)

# Quick visualization of submited window predictions

In [ ]:
from torch_points3d.datasets.segmentation.kitti360 import read_kitti360_window, ID2TRAINID
from torch_points3d.core.multimodal import MMData, ImageData
from torch_points3d.datasets.segmentation.kitti360 import KITTI360_NUM_CLASSES, INV_OBJECT_LABEL, OBJECT_COLOR, IGNORE
import os.path as osp

i_submission_file = 0

# Get the submission file from submission directory
_submission_dir = '/raid/dataset/pointcloud/data/kitti360/results/submissions/xyzrgb/2021-11-11_9-19-56'
path = sorted(glob.glob(_submission_dir + '/*.npy'))[i_submission_file]

# Add ignored class in black
class_names = [INV_OBJECT_LABEL[i] for i in range(KITTI360_NUM_CLASSES)] + ['ignored']
class_colors = np.append(OBJECT_COLOR, np.zeros((1, 3), dtype=np.uint8), axis=0)

# Recover window name and associated raw file
window_name = osp.splitext(osp.basename(path)[5:])[0]
idx_window = [osp.splitext(osp.basename(x))[0] for x in trainer._dataset.test_dataset[0].raw_3d_paths].index(window_name)
raw_window_path = trainer._dataset.test_dataset[0].raw_3d_paths[idx_window]

# Load labels
y = torch.from_numpy(ID2TRAINID[np.load(path)])

# Load raw data
data = read_kitti360_window(raw_window_path, xyz=True, rgb=True, remap=True)
data.y = y
data.mapping_index = torch.arange(data.num_nodes)

In [ ]:
mm_data = MMData(data=data, image=ImageData([SameSettingImageData()]))
visualize_mm_data(mm_data, figsize=1000, pointsize=3, voxel=0.5, show_2d=False, class_names=class_names, class_colors=class_colors)